In [9]:
!pip install beautifulsoup4
!pip install requests
!pip install requests_html
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 6.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 5.6 MB/s eta 0:00:0000:0100:01


In [20]:
from bs4 import BeautifulSoup
# from requests_html import HTMLSession

import pandas as pd

In [11]:
import requests

home_url = 'https://www.livefootballtickets.com/euro-cup-tickets.html'
country_url = 'https://www.livefootballtickets.com/euro-cup/albania-football-tickets.html'
match_url = 'https://www.livefootballtickets.com/fixtures/match-1-germany-v-a2-tickets-euro-2024.html'

html_text = requests.get(country_url).text

match_page_text = requests.get(match_url).text

print(match_page_text)



<!DOCTYPE html>
<html class="no-js" lang="en">
    <head><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="msvalidate.01" content="4E29BE86609662993923B68A064E1BCA" /><link href="/main/styles?v=1Awv-O0I-PHJnPjwWzyRq5r_1Zx71-_7nvtg4BTwFTM1" rel="stylesheet"/>
<script>
window.dataLayer = window.dataLayer || [];
window.dataLayer.push({
event: 'eec.detail',
ecommerce: {
detail: {
products: [{
'id':'37875','name':'MATCH 1 - Germany v Scotland','category':'Euro 2024','brand':'MATCH 1 - Germany'}] } } });
</script>
<!--Google Tag Manager -->
<script> (function(w, d, s, l, i){ w[l] = w[l] ||[]; w[l].push({'gtm.start': new Date().getTime(),event:'gtm.js'});var f = d.getElementsByTagName(s)[0], j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async=true;j.src= '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f); })
(window,document,'script','dataLayer','GTM-5F72PW');</script><!-- End Google Tag Mana

In [12]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_text, 'html.parser')

prices = soup.find_all("div", {"class": "ticket-price"})

In [31]:
pip install pandas_gbq

Note: you may need to restart the kernel to use updated packages.


In [46]:
import re
# import pandas as pd
import datetime
import pandas_gbq

project_id = 'ams-ticket-tracker'

soup = BeautifulSoup(requests.get(home_url).text, 'html.parser')

# Find all the links to country pages within the 'team-list' div
country_links = soup.select('div.team-list a.team-list-item')

# List to hold the URLs and the HTML content of each country page
country_pages_content = []

all_match_links = []

current_time = datetime.datetime.now()
table_name_suffix = current_time.strftime('%Y%m%d%H')

# Iterate through the list of country links
for link in country_links:
    country_url = link['href']
    country_name = link.get_text(strip=True)
    
    # Send a GET request to the country page
    country_response = requests.get(country_url)
    country_page_content = country_response.text

    # Store the URL and content in a dictionary, then add to the list
    country_pages_content.append({
        'country_name': country_name,
        'url': country_url,
        'html_content': country_page_content
    })

country_df = pd.DataFrame(country_pages_content)

country_table_name = f"raw.country_links_{table_name_suffix}"
pandas_gbq.to_gbq(country_df, country_table_name, project_id=project_id, if_exists='replace')

for country in country_pages_content:

    country_url = country['url']
    # Find all the 'a' elements within the 'h4' tags inside 'event-list-item-inner' divs

    country_soup = BeautifulSoup(requests.get(country_url).text, 'html.parser')   

    match_links = country_soup.select('.event-list-item-inner .event-info-name a')

    # Extract the 'href' attribute (the URL) from each link
    match_urls = [a['href'] for a in match_links]

    print(match_urls)

    matches_data = []
    ticket_data = []

    for match_url in match_urls:
        if match_url in all_match_links:
            pass
        else:
            all_match_links.append(match_url)

            match_soup = BeautifulSoup(requests.get(match_url).text, 'html.parser')
            venue = match_soup.find(attrs={"data-qa": "event-venue"}).get_text(strip=True)
            # Find the element with the data-qa attribute set to 'event-location'
            location = match_soup.find(attrs={"data-qa": "event-location"}).get_text(strip=True)
            tickets_available = match_soup.find(class_="tickets-available").span.get_text(strip=True)
            tickets = match_soup.select('li.ticket-row')

            # Find the element with the class 'box-header--title'
            title_element = match_soup.find(class_="box-header--title").get_text(strip=True)
            
            title_pattern = r'MATCH (\d+) - (.+?)v(.+) Tickets'
            title_match = re.search(title_pattern, title_element)

            # Define a regex pattern to extract the team codes from the URL
            url_pattern = r'match-\d+-(\w+)-v-(\w+)-tickets'
            url_matched = re.search(url_pattern, match_url)

            if title_match and url_matched:
                match_number = title_match.group(1)
                team_one = title_match.group(2).strip()
                team_two = title_match.group(3).strip()
                team_one_code = url_matched.group(1).upper()
                team_two_code = url_matched.group(2).upper()
                print(match_number, team_one, team_two, team_one_code, team_two_code)

            match_dict = {
                'match_url': match_url,
                'match_id': match_number,
                'venue': venue,
                'partition': 1,
                'home_country': team_one,
                'away_country': team_two,
                'home_country_id': team_one_code,
                'away_country_id': team_two_code
            }

            matches_data.append(match_dict)

            # print('M', match_number, home, away)

            for ticket in tickets:
                stadium_id = ticket['data-stadium-id']
                ticket_heading = ticket.find('h5', class_='ticket-heading').text
                ticket_price = ticket.find('div', class_='ticket-price').h4.text

                ticket_quantity_div = ticket.find('div', class_='ticket-quantity')
                select_element = ticket_quantity_div.find('select')
                option_values = [int(option['value']) for option in select_element.find_all('option')]
                ticket_quantities = max(option_values) if option_values else None
                
                ticket_category = soup.find('span', class_='ticket-info-details clearfix')
                ticket_dict = {
                    'match_id': match_number,
                    'stadium_id': stadium_id,
                    'tickets_available': tickets_available,
                    # 'game_id': match_number,
                    'ticket_heading': ticket_heading,
                    'category': ticket_category,
                    'ticket_price': ticket_price,
                    'ticket_quantities': ticket_quantities,
                    'home_country_id': team_one_code,
                    'away_country_id': team_two_code
                    }
                ticket_data.append(ticket_dict)

                # print(ticket_dict)
                
                # print(f"Stadium ID: {stadium_id}")
                # print()
                # print(f"Ticket Heading: {ticket_heading}")
                # print(f"Ticket Price: {ticket_price}")
                # # print(f"Highest Ticket Quantity: {highest_quantity}")
                # print("-----")

            


tickets_df = pd.DataFrame(ticket_data)
matches_df = pd.DataFrame(matches_data)

match_table_name = f"raw.match_links_{table_name_suffix}"
pandas_gbq.to_gbq(matches_df, match_table_name, project_id=project_id, if_exists='replace')

ticket_table_name = f"raw.ticket_prices_{table_name_suffix}"
pandas_gbq.to_gbq(tickets_df, ticket_table_name, project_id=project_id, if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 12483.05it/s]


['https://www.livefootballtickets.com/fixtures/match-4-b3-v-b4-tickets-euro-2024.html', 'https://www.livefootballtickets.com/fixtures/match-15-b2-v-b4-tickets-euro-2024.html', 'https://www.livefootballtickets.com/fixtures/match-27-b4-v-b1-tickets-euro-2024.html']
4 Italy Albania B3 B4
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 215.00
-----
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 217.00
-----
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 218.75
-----
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 218.76
-----
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 219.00
-----
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £225.63
-----
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 225.76
-----
Stadium ID: C_1930

Ticket Heading: CATEGORY 3 - NEUTRAL 

In [25]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "/Users/alexsmith/Programming/ams-ticket-tracker-267109157737.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

# Use the credentials to create a client
client = bigquery.Client(credentials=credentials)

In [ ]:
import pandas_gbq

import datetime
current_time = datetime.now()
table_name_suffix = current_time.strftime('%Y%m%d_%H')
# Assuming you have a DataFrame named 'df'

# Replace 'your_project_id' with your actual project ID
project_id = 'ams-ticket-tracker'

# Replace 'your_dataset.your_table_name' with your dataset and table name
table_id = 'your_dataset.your_table_name'

# Use the to_gbq function to upload the DataFrame
pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists='replace')

In [14]:
tickets = soup.select('li.ticket-row')

for ticket in tickets:
    stadium_id = ticket['data-stadium-id']
    ticket_heading = ticket.find('h5', class_='ticket-heading').text
    ticket_price = ticket.find('div', class_='ticket-price').h4.text
    ticket_quantities = [int(option['value']) for option in ticket.select('select.ticket-quantity option')]
    # highest_quantity = max(ticket_quantities)
    
    print(f"Stadium ID: {stadium_id}")
    print(f"Ticket Heading: {ticket_heading}")
    print(f"Ticket Price: {ticket_price}")
    # print(f"Highest Ticket Quantity: {highest_quantity}")
    print("-----")


In [15]:
soup = BeautifulSoup(requests.get(home_url).text, 'html.parser')
country_links = soup.select('div.team-list a.team-list-item')

